In [ ]:
true_file='/content/drive/My Drive/Projects_Colab/intents.json'

In [ ]:

import nltk;
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense , Dropout
import random
import json
import tensorflow as tf

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
with open(true_file) as f:
    data = json.load(f)

In [ ]:
data

{'intents': [{'context_set': '',
   'patterns': ['Hi',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up'],
   'responses': [' Hello!',
    'Good to see youuu bro!',
    'Hi ma, how can I help?'],
   'tag': 'greeting'},
  {'context_set': '',
   'patterns': ['see you',
    'See you later',
    'Goodbye',
    'I am Leaving',
    'Have a Good day'],
   'responses': ['Sad to see you go :(',
    'Talk to you later bro',
    'Goodbye! love you da'],
   'tag': 'goodbye'},
  {'context_set': '',
   'patterns': ['how old',
    'how old is ',
    'what is your age',
    'how old are you',
    'age?'],
   'responses': [' 19 years', '19 years young!'],
   'tag': 'age'},
  {'context_set': '',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Enda.',
    "I'm Enda!",
    "I'm Enda aka vishwa's bot."],
   'tag': 'name'},
  {'context_set': '',
   'patterns': ['Id like to buy something',
    '

In [ ]:
data['intents']

[{'context_set': '',
  'patterns': ['Hi',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day',
   'Whats up'],
  'responses': [' Hello!', 'Good to see youuu bro!', 'Hi ma, how can I help?'],
  'tag': 'greeting'},
 {'context_set': '',
  'patterns': ['see you',
   'See you later',
   'Goodbye',
   'I am Leaving',
   'Have a Good day'],
  'responses': ['Sad to see you go :(',
   'Talk to you later bro',
   'Goodbye! love you da'],
  'tag': 'goodbye'},
 {'context_set': '',
  'patterns': ['how old',
   'how old is ',
   'what is your age',
   'how old are you',
   'age?'],
  'responses': [' 19 years', '19 years young!'],
  'tag': 'age'},
 {'context_set': '',
  'patterns': ['what is your name',
   'what should I call you',
   'whats your name?'],
  'responses': ['You can call me Enda.',
   "I'm Enda!",
   "I'm Enda aka vishwa's bot."],
  'tag': 'name'},
 {'context_set': '',
  'patterns': ['Id like to buy something',
   'whats on the menu',
   'what do you reccommend?',
   'cou

In [ ]:
    words=[]
    labels=[]
    docs_x=[]
    docs_y=[]
    for intent in data['intents']:
        for pattern in intent['patterns']:
            wrds=nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent['tag'])
            
            if(intent['tag'] not in labels):
                labels.append(intent['tag'])
    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels= sorted(labels)
    
    training = []
    output = []
    with open("data.pickle","wb") as f:
        pickle.dump((words,labels,training,output),f)
    out_empty = [0 for _ in range(len(labels))]
    
    for x,doc in enumerate(docs_x):
        bags = []
        wrds = [stemmer.stem(w) for w in doc]
        for w in words:
            if w in wrds:
                bags.append(1)
            else:
                bags.append(0)
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1
        
        training.append(bags)
        output.append(output_row)
        
    training = np.array(training)
    output = np.array(output)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(None,len(training[0]))))

model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(16, activation='relu'))

model.add(tf.keras.layers.Dense(len(output[0]),activation='softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])


In [ ]:
model.fit(training,output,epochs=350,batch_size = 8,verbose=1)
model.save("model.h5")

Epoch 1/350
4/4 [==============================] - 0s 3ms/step - loss: 1.7989 - accuracy: 0.1538
Epoch 2/350
4/4 [==============================] - 0s 2ms/step - loss: 1.7741 - accuracy: 0.1538
Epoch 3/350
4/4 [==============================] - 0s 2ms/step - loss: 1.7444 - accuracy: 0.2308
Epoch 4/350
4/4 [==============================] - 0s 2ms/step - loss: 1.7837 - accuracy: 0.1923
Epoch 5/350
4/4 [==============================] - 0s 2ms/step - loss: 1.7729 - accuracy: 0.1538
Epoch 6/350
4/4 [==============================] - 0s 2ms/step - loss: 1.7477 - accuracy: 0.2692
Epoch 7/350
4/4 [==============================] - 0s 2ms/step - loss: 1.7489 - accuracy: 0.1154
Epoch 8/350
4/4 [==============================] - 0s 2ms/step - loss: 1.7661 - accuracy: 0.2692
Epoch 9/350
4/4 [==============================] - 0s 2ms/step - loss: 1.7968 - accuracy: 0.1923
Epoch 10/350
4/4 [==============================] - 0s 2ms/step - loss: 1.7290 - accuracy: 0.2692
Epoch 11/350
4/4 [===========

In [ ]:
def bag_of_words(s,words):
    bag=[0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i,w in enumerate(words):
            if w==se:
                bag[i]=1
    return bag

def chat():
    print("Enda Welcome (quit to stop)")
    while True:
        inp = input("You : ")
        if inp.lower()=='quit':
            break
        result=model.predict([bag_of_words(inp,words)])
        results_index = np.argmax(result)
        if result[0][results_index] > 0.7 :
            
            tag = labels[results_index]
            for tg in data['intents']:
                if tg['tag']==tag:
                    response = tg['responses']
            print(random.choice(response))
        else:
            print("Enda did not understand.Try again")

In [ ]:
chat()

Enda Welcome (quit to stop)
You : Hello
Good to see youuu bro!
You : What is your age
 19 years
You : Whats your name?
I'm Enda!
You : Goodbye
Enda did not understand.Try again
You : See you later
Sad to see you go :(


KeyboardInterrupt: ignored

In [ ]:
labels


['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']